In [3]:
pip install GraphQLClient

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install filesplit

Note: you may need to restart the kernel to use updated packages.


In [3]:
from graphqlclient import GraphQLClient
import pandas as pd
import numpy as np
import json
from fsplit.filesplit import Filesplit # Needed in order to split the files into smaller chunks
fs = Filesplit()
import os # needed for changing directories for files
from itertools import combinations #needed to get all combinations of pairs of tokennames

In [4]:
def intro():
    client = GraphQLClient('https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2')
    m=client.execute("""
    query IntrospectionQuery {
      __schema {
        
        queryType { name }
        mutationType { name }
        subscriptionType { name }
        types {
          ...FullType
        }
        directives {
          name
          description
          
          locations
          args {
            ...InputValue
          }
        }
      }
    }

    fragment FullType on __Type {
      kind
      name
      description
      fields(includeDeprecated: true) {
        name
        description
        args {
          ...InputValue
        }
        type {
          ...TypeRef
        }
        isDeprecated
        deprecationReason
      }
      inputFields {
        ...InputValue
      }
      interfaces {
        ...TypeRef
      }
      enumValues(includeDeprecated: true) {
        name
        description
        isDeprecated
        deprecationReason
      }
      possibleTypes {
        ...TypeRef
      }
    }

    fragment InputValue on __InputValue {
      name
      description
      type { ...TypeRef }
      defaultValue
    }

    fragment TypeRef on __Type {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
          ofType {
            kind
            name
            ofType {
              kind
              name
              ofType {
                kind
                name
                ofType {
                  kind
                  name
                  ofType {
                    kind
                    name
                  }
                }
              }
            }
          }
        }
      }
    }
    """)
    return m

In [5]:
a =intro()
print(a)

{"data":{"__schema":{"queryType":{"name":"Query"},"mutationType":null,"subscriptionType":{"name":"Subscription"},"types":[{"kind":"OBJECT","name":"Query","description":null,"fields":[{"name":"comptroller","description":"","args":[{"name":"id","description":"","type":{"kind":"NON_NULL","name":null,"ofType":{"kind":"SCALAR","name":"ID","ofType":null}},"defaultValue":null},{"name":"block","description":"The block at which the query should be executed. Can either be an `{ number:\nInt }` containing the block number or a `{ hash: Bytes }` value containing a\nblock hash. Defaults to the latest block when omitted.","type":{"kind":"INPUT_OBJECT","name":"Block_height","ofType":null},"defaultValue":null},{"name":"subgraphError","description":"Set to `allow` to receive data even if the subgraph has skipped over errors while syncing.","type":{"kind":"NON_NULL","name":null,"ofType":{"kind":"ENUM","name":"_SubgraphErrorPolicy_","ofType":null}},"defaultValue":"deny"}],"type":{"kind":"OBJECT","name":"

In [2]:
"""
from google.colab import drive
drive.mount('/content/drive')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
"""
# Comment these out since I am not using google drive to store the files

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n"

### Splitting the "swaps", "burns" and "mints" files into smaller files

I do this to be able to read and write to them faster (my laptop cannot handle loading the swaps file into memory)

Important to note that I have the jupyter notebook in the folder with swaps burns and mints folders

In [5]:
main_dir = os.getcwd() # Get current working directory, i.e. the location of the jupyter notebook
# ! important to run this before any kind of changing directories, since otherwise main_dir will be wrong
print(main_dir)

C:\Users\riho8\Documents\School\Internship\Uniswap


In [6]:
# Setting up the paths for the folders containing the main large files and then the folders containing the split files
# Note that without the "r" before the string, I got an error trying to access anything starting with a "b"
path_swaps = fr"{main_dir}\swaps\swaps_all_conc.csv"
path_burns = fr"{main_dir}\burns\burns_all_conc.csv"
path_mints = fr"{main_dir}\mints\mints_all_conc.csv"
dir_swaps = fr"{main_dir}\swaps\swaps_split"
dir_mints = fr"{main_dir}\mints\mints_split"
dir_burns = fr"{main_dir}\burns\burns_split"

dir_swaps_output = fr"{main_dir}\swaps\swaps_output"
dir_burns_output = fr"{main_dir}\burns\burns_output"
dir_mints_output = fr"{main_dir}\mints\mints_output"


os.chdir(dir_burns) #Debugging
print(os.getcwd())
os.chdir(main_dir)#return to main directory
print(os.getcwd())

C:\Users\riho8\Documents\School\Internship\Uniswap\burns\burns_split
C:\Users\riho8\Documents\School\Internship\Uniswap


Breaking the big files down into smaller ones

In [7]:
# Definition of splitting function, for more explanation see "https://pypi.org/project/filesplit/"
def split(path_main_file, dir_output):
    """
    path_main_file: Path to the file to be split
    dir_output: Directory of the folder where the split files will be saved
    
    Splits the files into 10mb chunks
    """
    fs.split(file=path_main_file, split_size=500000, output_dir=dir_output, newline = True, include_header = True)


In [6]:
#split(path_burns,dir_burns)
split(path_swaps,dir_swaps)
#split(path_mints,dir_mints)
print("done")

done


# To further optimise the code:
I would have to sort the files by pair_id, so that when we run the query for all pair_id's you do not have to search through all files for anything

# Importation des données

There are 3 different datasets: one for the swaps , one for the mint events and one for the burn events


Riho Note: I don't need the following three cells due to the files being saved on my computer 

In [ ]:
# Download du fichier swaps

"""
downloaded = drive.CreateFile({'id': '1P554jz7Rkz0_7PLjEKkHcu8nKRpPaUwf'})

downloaded.GetContentFile('swaps_all_conc.csv')
import pandas as pd
swaps_all_conc = pd.read_csv('swaps_all_conc.csv')
"""

In [ ]:
# Download du fichier burns
"""
downloaded = drive.CreateFile({'id': '1fkrVZdfMlofa5Cx6Qy554Cn4mzi8cHwa'})

downloaded.GetContentFile('burns_all_conc.csv')
import pandas as pd
burn_all_conc = pd.read_csv('burns_all_conc.csv')
"""

In [ ]:
# Download du fichier mints
"""
downloaded = drive.CreateFile({'id': '1-7qnHVij21vINB9ptMoOZwFCzJPGY2l7'})

downloaded.GetContentFile('mints_all_conc.csv')
import pandas as pd
mints_all_conc = pd.read_csv('mints_all_conc.csv')
"""

# Les Queries

In [8]:
#The query for the swap events
#I run the query for a specific pair_id and transaction_id as (pair_id,transaction_id) is a way to uniquely idenitfy transactions 
#(two transactions may have the same transaction id but are from differents pair and so if you don't specify the pair, the query will give you several results)
def swap(i,j):
    client = GraphQLClient('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2')
    m=client.execute('''
    {
     swaps( where:
     { pair: "'''+str(i)+'''" ,transaction:"'''+str(j)+'''"}
    ) {
         pair {
           token0 {
             name
           }
           token1 {
             name
           }
         }
         timestamp
         amount0In
         amount0Out
         amount1In
         amount1Out
         amountUSD
         transaction{
         id
         blockNumber
         }
         logIndex
     }
    }
    ''')
    return(m)

In [9]:
#The query for the mint events
def mint(i,j):
    client = GraphQLClient('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2')
    m=client.execute('''
    {
     mints( where:
     { pair: "'''+str(i)+'''" ,transaction:"'''+str(j)+'''"}
    ) {
         pair {
           token0 {
             name
           }
           token1 {
             name
           }
         }
         amount0
         amount1
         amountUSD
         timestamp
         transaction{
         id
         blockNumber
         }
         logIndex
     }
    }
    ''')
    return(m)

In [10]:
#The query for the burn events
def burn(i,j):
    client = GraphQLClient('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2')
    m=client.execute('''
    {
     burns( where:
     { pair: "'''+str(i)+'''" ,transaction:"'''+str(j)+'''"}
    ) {
         pair {
           token0 {
             name
           }
           token1 {
             name
           }
         }
         amount0
         amount1
         amountUSD
         timestamp
         transaction{
         id
         blockNumber
         }
         logIndex
     }
    }
    ''')
    return(m)

# Extractions des données: blocknb & log_index

Riho Note: Modified the code so that it works on smaller files and loops over all the files

In [11]:
def get_files_in_dir(directory):
    """
    goes to a directory and returns an array of all the filenames in the direcory
    """
    global main_dir # get the global main directory in order to be able to return to it latre
    os.chdir(directory)
    files = os.listdir()
    os.chdir(main_dir)
    return files

def save_df_to_dir(dataframe,directory,filename):
    """
        saves dataframe to specific directory
    """
    global main_dir
    os.chdir(directory)
    dataframe.to_csv(f'{filename}.csv') 
    os.chdir(main_dir)

In [12]:
# I concatenate the path to the file with the filename to get the full filepath

swaps_filepaths = [fr"{dir_swaps}\\{i}" for i in get_files_in_dir(dir_swaps)]
mints_filepaths = [fr"{dir_mints}\\{i}" for i in get_files_in_dir(dir_mints)]
burns_filepaths = [fr"{dir_burns}\\{i}" for i in get_files_in_dir(dir_burns)]

### commented out the code below since I do not load the entire dataset at once, I added this part of the code into the data extraction cells

In [16]:
#Here I run the queries for a specific pair_id but the idea is to do it for all the transactions of the dataset (not only one pair)
#swaps11=swaps_all_conc[ (swaps_all_conc["token1_name"]=="Tether USD")  & (swaps_all_conc["token0_name"]=="Wrapped Ether")]

In [20]:
#Just to try

for count,file in enumerate(swaps_filepaths): # Added this to loop over the filenames
    swap_info = []
    pb_swap = []# In case, the code failed to query the graph for a specific transaction (because sometimes it fails but when you try again, it works), but it should be empty as it will try until it gets something
    i = 0 
    print(file) 
    dataframe = pd.read_csv(file) # read the current file
    swaps11 = dataframe[ (dataframe["token1_name"]=="Tether USD")  & (dataframe["token0_name"]=="Wrapped Ether")]
    if not swaps11.empty:
        i=0
        l = swaps11["transaction_id"].iloc[i] #transaction_id
        k = swaps11["pair_id"].iloc[i]#Pair_id
        m = swap(k,l)
        print(i,m)
        x = [(k,l,info['transaction']["id"],info['transaction']["blockNumber"],info['logIndex'],
                         info['timestamp'],info['pair']['token0']["name"],info['pair']['token1']["name"],info['amount0In'],info['amount0Out'],info['amount1In'],info['amount1Out'],info['amountUSD']) for info in json.loads(m)['data']['swaps']]          

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_10.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_100.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1000.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1001.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1002.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1003.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1004.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1005.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1006.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_spl

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_108.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1080.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1081.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1082.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1083.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1084.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1085.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1086.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1087.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1088.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swap

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1159.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_116.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1160.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1161.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1162.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1163.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1164.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1165.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1166.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1167.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swap

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1243.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1244.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1245.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1246.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1247.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1248.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1249.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_125.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1250.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1251.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swap

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1325.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1326.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1327.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1328.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1329.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_133.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1330.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1331.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1332.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1333.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swap

...

0 {"data":{"swaps":[{"amount0In":"0.1","amount0Out":"0","amount1In":"0","amount1Out":"38.570006","amountUSD":"38.35906902958602782217413909671367","logIndex":"217","pair":{"token0":{"name":"Wrapped Ether"},"token1":{"name":"Tether USD"}},"timestamp":"1600390320","transaction":{"blockNumber":"10883078","id":"0x11cbd8d47117bbd41efb6fe9f6bdae4960f2fe5d4fdf37d5e9e04b542659d9a1"}}]}}
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_9452.csv
0 {"data":{"swaps":[{"amount0In":"0","amount0Out":"4.299240743253698505","amount1In":"1657","amount1Out":"0","amountUSD":"1648.226734873215706905860032462054","logIndex":"263","pair":{"token0":{"name":"Wrapped Ether"},"token1":{"name":"Tether USD"}},"timestamp":"1600403339","transaction":{"blockNumber":"10884009","id":"0xc546a9ccb2fa87b36db349ac8e62e088f532a3b3e2791f3c18ff068a17d8ebe0"}}]}}
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_9453.csv
0 {"data":{"swaps":[{"amount0In":"0"

0 {"data":{"swaps":[{"amount0In":"0.05007511043417389","amount0Out":"0","amount1In":"0","amount1Out":"18.967096","amountUSD":"18.90172248377746460246673617141191","logIndex":"7","pair":{"token0":{"name":"Wrapped Ether"},"token1":{"name":"Tether USD"}},"timestamp":"1600595081","transaction":{"blockNumber":"10898430","id":"0xd5cd4dc309566e6b74b9b5f4ae4514befa53e9b278e34e437cd3b667768dd319"}}]}}
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_947.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_9470.csv
0 {"data":{"swaps":[{"amount0In":"1.769207255783471648","amount0Out":"0","amount1In":"0","amount1Out":"656.233818","amountUSD":"655.821403443604726701094092766434","logIndex":"144","pair":{"token0":{"name":"Wrapped Ether"},"token1":{"name":"Tether USD"}},"timestamp":"1600607038","transaction":{"blockNumber":"10899380","id":"0x483a1de43ab2357dbabd7996676f6e18808598fdb279a1cdef7f8f028171c313"}}]}}
C:\Users\riho8\Doc

0 {"data":{"swaps":[{"amount0In":"0","amount0Out":"7.245544136041918312","amount1In":"2491.197408","amount1Out":"0","amountUSD":"2474.632884510147911812747714483783","logIndex":"215","pair":{"token0":{"name":"Wrapped Ether"},"token1":{"name":"Tether USD"}},"timestamp":"1600780605","transaction":{"blockNumber":"10912503","id":"0x09f2791048750029fb77fb4c34b85e3490f988d2c50b479de251f2a6d611b1b9"}}]}}
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_9488.csv
0 {"data":{"swaps":[{"amount0In":"1.468492063292140491","amount0Out":"0","amount1In":"0","amount1Out":"500","amountUSD":"498.6409299524713234123016826366147","logIndex":"132","pair":{"token0":{"name":"Wrapped Ether"},"token1":{"name":"Tether USD"}},"timestamp":"1600789204","transaction":{"blockNumber":"10913160","id":"0x81c976a25e29eb178599712732f8c69a3d8e2dffac3254f16d176f139fa499fa"}}]}}
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_9489.csv
0 {"data":{"swaps"

0 {"data":{"swaps":[{"amount0In":"0","amount0Out":"2.214363866072828143","amount1In":"750","amount1Out":"0","amountUSD":"746.0466041155446007607714748305595","logIndex":"122","pair":{"token0":{"name":"Wrapped Ether"},"token1":{"name":"Tether USD"}},"timestamp":"1600957262","transaction":{"blockNumber":"10925840","id":"0xa459dc5eb08a49d335f0609fd9db895b2011b6c5cde5a1970e2992f329bd7b77"}}]}}
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_9505.csv
0 {"data":{"swaps":[{"amount0In":"0","amount0Out":"5.754731961386707483","amount1In":"2000","amount1Out":"0","amountUSD":"1988.768172965944517285787479181944","logIndex":"248","pair":{"token0":{"name":"Wrapped Ether"},"token1":{"name":"Tether USD"}},"timestamp":"1600967024","transaction":{"blockNumber":"10926558","id":"0x97271e97b1ecdf528f784bb0c611b55336136c21e59017256bf2337523326cf5"}}]}}
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_9506.csv
0 {"data":{"swaps":[{"amo

KeyboardInterrupt: 

In [11]:
os.chdir(dir_swaps)
df = pd.read_csv(os.listdir()[0])
#print(df.head())
os.chdir(main_dir)
df_test = pd.read_csv(swaps_filepaths[0])
print(df_test)

      Unnamed: 0  Unnamed: 0.1                                     pair_id  \
0              0             0  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   
1              1             1  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   
2              2             2  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   
3              3             3  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   
4              4             4  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   
...          ...           ...                                         ...   
1635        1635          1635  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   
1636        1636          1636  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   
1637        1637          1637  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   
1638        1638          1638  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   
1639        1639          1639  0xbb2b8038a1640196fbe3e38816f3e67cba72d940   

                     date  token0_name    token1_name  amount0I

### REWRITTEN VERSION OF THE DATA EXTRACTION CODE
noting is taken away, I have only added a for loop that loops over all the files in the filenames arrays

So I figured that you can get all the transactions just by passing pair ID and transaction ID to the query, so we do not have any need to get the token names or ID's. We can extract specific pairs by specifiying their pair ID. I have written this code with that in mind

In [13]:
pair_id = "0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852" #example pair_id, change this to change which pair we are extracting

In [ ]:
# The files aren't stored in alphabetical order in the swaps_filepaths list, so it might not print the files in increasing order

"""
!!! Not yet implemented, put everything into this for loop to loop over all token combos

for token_combo in token_combinations:
    token1 = token_combo[0]
    token2 = token_combo[1]
"""



for count,file in enumerate(swaps_filepaths): # Added this to loop over the filenames
    swap_info = []
    pb_swap = []# In case, the code failed to query the graph for a specific transaction (because sometimes it fails but when you try again, it works), but it should be empty as it will try until it gets something
    i = 0 
    print(file) 
    dataframe = pd.read_csv(file) # read the current file
    # Rewrite the following line to loop over all the tokennames, most likely will add an extra loop
    swaps11 = dataframe.loc[dataframe["pair_id"] == pair_id] #get everything where the pair_id matches the current pair we are looking at 
    if not swaps11.empty: #extra check to make sure that the specified pair is actually in the dataframe
        while i < swaps11.shape[0] :
            l = swaps11["transaction_id"].iloc[i] #transaction_id
            k = pair_id #Pair_id, don't have to locate since all pairs already have this
            try:
                m = swap(k,l)
                if m != '{"data":{"swaps":[]}}' :#until there is no more data, !Infinite loop, changed while to if
                    if i > 2000:
                        print(m)
                    x = [(k,l,info['transaction']["id"],info['transaction']["blockNumber"],info['logIndex'],# I store both l and transaction_id so that I'll check that I've the right data
                         info['timestamp'],info['pair']['token0']["name"],info['pair']['token1']["name"],info['amount0In'],info['amount0Out'],info['amount1In'],info['amount1Out'],info['amountUSD'])for info in json.loads(m)['data']['swaps']]#store the results in a list
                    swap_info =  swap_info + x #add the result of the query to the previous ones
                    print(i,len(x)) #print i to know how many obervsations left (as we go through all the transactions of the dataset), len(x) to make sure that is equal to 1 i.e the tuple(pair_id,transaction_id) 
                    #allows us to uniquely identify a given transaction of our dataset 
                    i = i+1 # to go to the next transaction of the original dataset to run then the query for it.
            except:
                print(m)
                pb_swap = pb_swap+[(i,l)]
        ### !!! Changed the saving to file implementation
        #from list to df
        df1 = pd.DataFrame(swap_info)
        #rename the columns
        df1.columns = ["pair_id","trans_id","trans_id2","blockNumber","log_index",'date','token0_name','token1_name',"amount0In","amount0Out","amount1In","amount1Out","amountUSD"] #name the columns
        #export the dataset
        save_df_to_dir(df1,dir_swaps_output,f"log_index_swap_usdt_eth_{pair_id}_{count}.csv")



    

####from google.colab import files
#export the dataset
####df1.to_csv('log_index_swap_usdt_eth.csv') 
####!cp log_index_swap_usdt_eth.csv "drive/My Drive/data/log_index_swap_usdt_eth.csv" #and here

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_10.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_100.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1000.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1001.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1002.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1003.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1004.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1005.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1006.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_spl

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1078.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1079.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_108.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1080.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1081.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1082.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1083.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1084.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1085.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1086.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swap

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1161.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1162.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1163.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1164.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1165.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1166.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1167.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1168.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1169.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_117.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swap

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1245.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1246.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1247.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1248.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1249.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_125.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1250.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1251.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1252.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1253.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swap

C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1332.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1333.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1334.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1335.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1336.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1337.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1338.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1339.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_134.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swaps_split\\swaps_all_conc_1340.csv
C:\Users\riho8\Documents\School\Internship\Uniswap\swaps\swap

...

1070 1
1071 1
1072 1
1073 1
1074 1
1075 1
1076 1
1077 1
1078 1
1079 1
1080 1
1081 1
1082 1
1083 1
1084 1
1085 1
1086 1
1087 1
1088 1
1089 1
1090 1
1091 1
1092 1
1093 1
1094 1
1095 1
1096 1
1097 1
1098 1
1099 1
1100 1
1101 1
1102 1
1103 1
1104 1
1105 1
1106 1
1107 1
1108 1
1109 1
1110 1
1111 1
1112 1
1113 1
1114 1
1115 1
1116 1
1117 1
1118 1
1119 1
1120 1
1121 1
1122 1
1123 1
1124 1
1125 1
1126 1
1127 1
1128 1
1129 1
1130 1
1131 1
1132 1
1133 1
1134 1
1135 1
1136 1
1137 1
1138 1
1139 1
1140 1
1141 1
1142 1
1143 1
1144 1
1145 1
1146 1
1147 1
1148 1
1149 1
1150 1
1151 1
1152 1
1153 1
1154 1
1155 1
1156 1
1157 1
1158 1
1159 1
1160 1
1161 1
1162 1
1163 1
1164 1
1165 1
1166 1
1167 1
1168 1
1169 1
1170 1
1171 1
1172 1
1173 1
1174 1
1175 1
1176 1
1177 1
1178 1
1179 1
1180 1
1181 1
1182 1
1183 1
1184 1
1185 1
1186 1
1187 1
1188 1
1189 1
1190 1
1191 1
1192 1
1193 1
1194 1
1195 1
1196 1
1197 1
1198 1
1199 1
1200 1
1201 1
1202 1
1203 1
1204 1
1205 1
1206 1
1207 1
1208 1
1209 1
1210 1
1211 1
1212 1

737 1
738 1
739 1
740 1
741 1
742 1
743 1
744 1
745 1
746 1
747 1
748 1
749 1
750 1
751 1
752 1
753 1
754 1
755 1
756 1
757 1
758 1
759 1
760 1
761 1
762 1
763 1
764 1
765 1
766 1
767 1
768 1
769 1
770 1
771 1
772 1
773 1
774 1
775 1
776 1
777 1
778 1
779 1
780 1
781 1
782 1
783 1
784 1
785 1
786 1
787 1
788 1
789 1
790 1
791 1
792 1
793 1
794 1
795 1
796 1
797 1
798 1
799 1
800 1
801 1
802 1
803 1
804 1
805 1
806 1
807 1
808 1
809 1
810 1
811 1
812 1
813 1
814 1
815 1
816 1
817 1
818 1
819 1
820 1
821 1
822 1
823 1
824 1
825 1
826 1
827 1
828 1
829 1
830 1
831 1
832 1
833 1
834 1
835 1
836 1
837 1
838 1
839 1
840 1
841 1
842 1
843 1
844 1
845 1
846 1
847 1
848 1
849 1
850 1
851 1
852 1
853 1
854 1
855 1
856 1
857 1
858 1
859 1
860 1
861 1
862 1
863 1
864 1
865 1
866 1
867 1
868 1
869 1
870 1
871 1
872 1
873 1
874 1
875 1
876 1
877 1
878 1
879 1
880 1
881 1
882 1
883 1
884 1
885 1
886 1
887 1
888 1
889 1
890 1
891 1
892 1
893 1
894 1
895 1
896 1
897 1
898 1
899 1
900 1
901 1
902 1
903 

394 1
395 1
396 1
397 1
398 1
399 1
400 1
401 1
402 1
403 1
404 1
405 1
406 1
407 1
408 1
409 1
410 1
411 1
412 1
413 1
414 1
415 1
416 1
417 1
418 1
419 1
420 1
421 1
422 1
423 1
424 1
425 1
426 1
427 1
428 1
429 1
430 1
431 1
432 1
433 1
434 1
435 1
436 1
437 1
438 1
439 1
440 1
441 1
442 1
443 1
444 1
445 1
446 1
447 1
448 1
449 1
450 1
451 1
452 1
453 1
454 1
455 1
456 1
457 1
458 1
459 1
460 1
461 1
462 1
463 1
464 1
465 1
466 1
467 1
468 1
469 1
470 1
471 1
472 1
473 1
474 1
475 1
476 1
477 1
478 1
479 1
480 1
481 1
482 1
483 1
484 1
485 1
486 1
487 1
488 1
489 1
490 1
491 1
492 1
493 1
494 1
495 1
496 1
497 1
498 1
499 1
500 1
501 1
502 1
503 1
504 1
505 1
506 1
507 1
508 1
509 1
510 1
511 1
512 1
513 1
514 1
515 1
516 1
517 1
518 1
519 1
520 1
521 1
522 1
523 1
524 1
525 1
526 1
527 1
528 1
529 1
530 1
531 1
532 1
533 1
534 1
535 1
536 1
537 1
538 1
539 1
540 1
541 1
542 1
543 1
544 1
545 1
546 1
547 1
548 1
549 1
550 1
551 1
552 1
553 1
554 1
555 1
556 1
557 1
558 1
559 1
560 

42 1
43 1
44 1
45 1
46 1
47 1
48 1
49 1
50 1
51 1
52 1
53 1
54 1
55 1
56 1
57 1
58 1
59 1
60 1
61 1
62 1
63 1
64 1
65 1
66 1
67 1
68 1
69 1
70 1
71 1
72 1
73 1
74 1
75 1
76 1
77 1
78 1
79 1
80 1
81 1
82 1
83 1
84 1
85 1
86 1
87 1
88 1
89 1
90 1
91 1
92 1
93 1
94 1
95 1
96 1
97 1
98 1
99 1
100 1
101 1
102 1
103 1
104 1
105 1
106 1
107 1
108 1
109 1
110 1
111 1
112 1
113 1
114 1
115 1
116 1
117 1
118 1
119 1
120 1
121 1
122 1
123 1
124 1
125 1
126 1
127 1
128 1
129 1
130 1
131 1
132 1
133 1
134 1
135 1
136 1
137 1
138 1
139 1
140 1
141 1
142 1
143 1
144 1
145 1
146 1
147 1
148 1
149 1
150 1
151 1
152 1
153 1
154 1
155 1
156 1
157 1
158 1
159 1
160 1
161 1
162 1
163 1
164 1
165 1
166 1
167 1
168 1
169 1
170 1
171 1
172 1
173 1
174 1
175 1
176 1
177 1
178 1
179 1
180 1
181 1
182 1
183 1
184 1
185 1
186 1
187 1
188 1
189 1
190 1
191 1
192 1
193 1
194 1
195 1
196 1
197 1
198 1
199 1
200 1
201 1
202 1
203 1
204 1
205 1
206 1
207 1
208 1
209 1
210 1
211 1
212 1
213 1
214 1
215 1
216 1
217 1
21

1358 1
1359 1
1360 1
1361 1
1362 1
1363 1
1364 1
1365 1
1366 1
1367 1
1368 1
1369 1
1370 1
1371 1
1372 1
1373 1
1374 1
1375 1
1376 1
1377 1
1378 1
1379 1
1380 1
1381 1
1382 1
1383 1
1384 1
1385 1
1386 1
1387 1
1388 1
1389 1
1390 1
1391 1
1392 1
1393 1
1394 1
1395 1
1396 1
1397 1
1398 1
1399 1
1400 1
1401 1
1402 1
1403 1
1404 1
1405 1
1406 1
1407 1
1408 1
1409 1
1410 1
1411 1
1412 1
1413 1
1414 1
1415 1
1416 1
1417 1
1418 1
1419 1
1420 1
1421 1
1422 1
1423 1
1424 1
1425 1
1426 1
1427 1
1428 1
1429 1
1430 1
1431 1
1432 1
1433 1
1434 1
1435 1
1436 1
1437 1
1438 1
1439 1
1440 1
1441 1
1442 1
1443 1
1444 1
1445 1
1446 1
1447 1
1448 1
1449 1
1450 1
1451 1
1452 1
1453 1
1454 1
1455 1
1456 1
1457 1
1458 1
1459 1
1460 1
1461 1
1462 1
1463 1
1464 1
1465 1
1466 1
1467 1
1468 1
1469 1
1470 1
1471 1
1472 1
1473 1
1474 1
1475 1
1476 1
1477 1
1478 1
1479 1
1480 1
1481 1
1482 1
1483 1
1484 1
1485 1
1486 1
1487 1
1488 1
1489 1
1490 1
1491 1
1492 1
1493 1
1494 1
1495 1
1496 1
1497 1
1498 1
1499 1
1500 1

In [61]:
# DO the same for the mint dataset
for count,file in enumerate(mints_filepaths):
    mint_info = []
    pb_swap = []
    i = 0
    dataframe = pd.read_csv(file)
    mints1 = dataframe.loc[dataframe["pair_id"] == pair_id]
    if not mints1.empty: #extra check to make sure that the specified pair is actually in the dataframe
        while i < mints1.shape[0] :
            l = mints1["transaction_id"].iloc[i]
            k = pair_id
            try:
                m = mint(k,l)
                if m != '{"data":{"swaps":[]}}' :#until there is no more data, !Infinite loop, changed while to if
                    if i > 2000:
                        print(m)
                    x = [(k,l,info['transaction']["id"],info['transaction']["blockNumber"],info['logIndex'],info['amount0'],info['amount1'],info['amountUSD'],info['timestamp'],info['pair']['token0']["name"],info['pair']['token1']["name"])for info in json.loads(m)['data']['mints']]
                    mint_info = mint_info+x
                    print(i,len(x))
                    i = i+1
            except:
                print((i,l))
                pb_swap = pb_swap+[(i,l)]
        df1 = pd.DataFrame(mint_info)
        df1.columns = ["pair_id","trans_id","trans_id2","blockNumber","log_index","amount0","amount1","amountUSD",'date','token0_name','token1_name'] #Rename the columns
        #export the dataset
        save_df_to_dir(df1,dir_mints_output,f"log_index_mint_usdt_eth_{pair_id}_{count}.csv")

NameError: name 'mints1' is not defined

In [ ]:
# DO the same for the burn dataset
for count,file in enumerate(mints_filepaths):
    burn_info = []
    pb_swap = []
    i = 0
    dataframe = pd.read_csv(file)
    burn1 = dataframe.loc[dataframe["pair_id"] == pair_id]
    if not burn1.empty:
        while i < burn1.shape[0] :
            l=burn1["transaction_id"].iloc[i]
            k=pair_id
            try:
                if m != '{"data":{"swaps":[]}}' :
                    m = burn(k,l)
                    x= [(k,l,info['transaction']["id"],info['transaction']["blockNumber"],info['logIndex'],info['amount0'],info['amount1'],info['amountUSD'],info['timestamp'],info['pair']['token0']["name"],info['pair']['token1']["name"])for info in json.loads(m)['data']['burns']]
                    burn_info = burn_info+ x
                    print(i,len(x)
                    i = i+1
            except:
                print((i,l))
                pb_swap = pb_swap+[(i,l)]

        df1 = pd.DataFrame(burn_info)
        df1.columns = ["pair_id","trans_id","trans_id2","blockNumber","log_index","amount0","amount1","amountUSD",'date','token0_name','token1_name'] #Rename the columns
        save_df_to_dir(df1,dir_burns_output,f"log_index_burn{pair_id}_{count}.csv")